# Quickstart
First let import the modules to work with.  Some functionalities of **TOFF** may not be available on PyPi. This could be because the new version is not yet release, but in brief it is going to be. You could install directly from the repo if some problem pops up. Just paste the following in a code cell:
```bash
! pip install git+https://github.com/ale94mleon/TOFF.git@main
```

## As module

In [29]:
from toff import Parameterize
from rdkit import Chem
import tempfile, os, yaml

In [30]:
# We will use this temporal directory, you could change it in your case to, for example wd = '.'
tmp_dir = tempfile.TemporaryDirectory()
wd = tmp_dir.name

### Initialize the class
The first steep is to create an instance of the class [Parameterize.__init__](https://toff.readthedocs.io/en/latest/source/modules/utils.html#toff.utils.Parameterize.__init__) (follow the link to know all possible arguments to initialize the class). For now we will use the default parameters, we will only specify the `out_dir`.


In [31]:
parameterizer = Parameterize(out_dir=wd)
parameterizer

Parameterize(force_field_code = openff_unconstrained-2.0.0.offxml, ext_types = [top pdb gro], hmr_factor = None, overwrite = False, out_dir = /tmp/tmp7ix2f0nu)

### call the class
You could see what ara all the arguments to specify here [Parameterize.__call__](https://toff.readthedocs.io/en/latest/source/modules/utils.html#toff.utils.Parameterize.__call__).
Becasue we kno that the molecule doe not have a conformation, then we set `gen_conformer=True`

In [32]:
parameterizer(Chem.MolFromSmiles('CCO'), gen_conformer=True)
os.listdir(wd)

Charges will be corrected: partial_charge - formal_charge = -0.002
After correction: partial_charge - formal_charge = 0.0.


['MOL.gro', 'MOL.pdb', 'MOL.top']

### Take a look on the .top file

In [33]:
with open(os.path.join(wd, 'MOL.top'), 'r') as f:
    text = f.read()
print(text)

;
;   File /tmp/tmp7ix2f0nu/MOL.top  was generated
;   By user: ale (1000)
;   On host: ale-MS-7B86
;   At date: Thu. November  4 18:09:16 2022
;
;   This is a standalone topology file
;
;   Created by:
;   ParmEd:       ipykernel_launcher.py, VERSION 3.4.3
;   Executable:   ipykernel_launcher.py
;   Library dir:  /usr/local/gromacs/share/gromacs/top
;   Command line:
;     /home/ale/anaconda3/envs/toff/lib/python3.10/site-packages/ipykernel_launcher.py -f /home/ale/.local/share/jupyter/runtime/kernel-47679357-03c6-4c4d-a25d-ed6ca5de61a9.json
;

[ defaults ]
; nbfunc        comb-rule       gen-pairs       fudgeLJ fudgeQQ
1               2               no              1            0.83333333  

[ atomtypes ]
; name    at.num    mass    charge ptype  sigma      epsilon
C1             6  12.010780  0.00000000  A     0.33795318     0.45538912
O1             8  15.999430  0.00000000  A       0.299716     0.87643726
H1             1   1.007947  0.00000000  A     0.26445434    0.066021356
H2

We could reuse the instance of Parameterize and used in other molecule

In [34]:
parameterizer(
    input_mol=Chem.MolFromSmiles('NCCCC'),
    mol_resi_name='LIG',
    gen_conformer=True,    
)
os.listdir(wd)

Charges will be corrected: partial_charge - formal_charge = 0.003
After correction: partial_charge - formal_charge = 0.0.


['LIG.top', 'MOL.gro', 'MOL.pdb', 'LIG.pdb', 'MOL.top', 'LIG.gro']

We would like to use [Hydrogen Mass Repartitioning](https://pubs.acs.org/doi/10.1021/ct5010406). This allows to increases the integration time step from 2 fs (usually) to 4fs. Here we have to create a new instance of Parameterize and set the `hmr_factor`. We will use 3, which means that the mas off the hydrogens atoms will increase in a factor of 3 using the mass of the heavy atoms attached to them.

In [35]:
parameterizer_hmr = Parameterize(out_dir=wd, hmr_factor=3)
parameterizer_hmr

Parameterize(force_field_code = openff_unconstrained-2.0.0.offxml, ext_types = [top pdb gro], hmr_factor = 3, overwrite = False, out_dir = /tmp/tmp7ix2f0nu)

In [36]:
parameterizer_hmr(
    input_mol=Chem.MolFromSmiles('CCF'),
    mol_resi_name='HMR',
    gen_conformer=True,    
)
os.listdir(wd)

Charges will be corrected: partial_charge - formal_charge = 0.001
After correction: partial_charge - formal_charge = -0.0.


['LIG.top',
 'MOL.gro',
 'HMR.top',
 'HMR.gro',
 'MOL.pdb',
 'LIG.pdb',
 'MOL.top',
 'LIG.gro',
 'HMR.pdb']

## As command line
We need to first create a yaml configuration file that is nothing else that a more human-readable python dictionary. The content of this file is all the variables needed for initialize and call the parameterize class. The only mandatory argument is `input_mol` and in this case must be a path to a file with the following extensions: smi, inchi, mol, mol2. In the case of smi and inchi only will be redded the first line of the file and must be a valid SMILES or InChi strings respectively.

In [37]:
# First let save a molecule
Chem.MolToMolFile(Chem.MolFromSmiles('CCCBr'), os.path.join(wd, 'my_file.mol'))
os.listdir(wd)


['LIG.top',
 'MOL.gro',
 'HMR.top',
 'HMR.gro',
 'MOL.pdb',
 'LIG.pdb',
 'MOL.top',
 'my_file.mol',
 'LIG.gro',
 'HMR.pdb']

In [38]:
config = {
    'force_field_code': 'openff-1.3.1.offxml', # by default is openff_unconstrained-2.0.0.offxml
    'ext_types': ['mol3', 'top'], # by default is ['top', 'pdb', 'gro']. And ypou could pick from: ‘pdb’, ‘pqr’, ‘cif’,’pdbx’, ‘parm7’, ‘prmtop’, ‘psf’, ‘top’, ‘gro’, ‘mol2’, ‘mol3’, ‘crd’, ‘rst7’, ‘inpcrd’, ‘restrt’, ‘ncrst’ 
    'hmr_factor': 2.5, # by default None
    'overwrite': True, # by default False
    'out_dir': os.path.join(wd, 'cmd'), # by default '.'
    'input_mol': os.path.join(wd, 'my_file.mol'), # The only parameter that MUST BE PROVIDED!
    'mol_resi_name': 'CMD', # by default 'MOL'
    'gen_conformer': True, # by default False
}

In [39]:
with open(os.path.join(wd, 'config.yml'), 'w') as cf:
    yaml.dump(config, cf)

print(f"This is how it looks the wd: {os.listdir(wd)}")

with open(os.path.join(wd, 'config.yml'), 'r') as cfy:
    text = cfy.read()
print("===========config.yml=======================")
print(text)
print("============================================")
# let's also get the path to the config.yml file to passed to the command line.
print(f"Path to the config file: {os.path.join(wd, 'config.yml')}")

This is how it looks the wd: ['LIG.top', 'MOL.gro', 'HMR.top', 'HMR.gro', 'MOL.pdb', 'LIG.pdb', 'MOL.top', 'my_file.mol', 'LIG.gro', 'config.yml', 'HMR.pdb']
===========config.yml=======================
ext_types:
- mol3
- top
force_field_code: openff-1.3.1.offxml
gen_conformer: true
hmr_factor: 2.5
input_mol: /tmp/tmp7ix2f0nu/my_file.mol
mol_resi_name: CMD
out_dir: /tmp/tmp7ix2f0nu/cmd
overwrite: true

Path to the config file: /tmp/tmp7ix2f0nu/config.yml


In [41]:
# Change the path properly. Yur temporal path is printed in the cell before and for sure it will not be the same as mine.

! parameterize /tmp/tmp7ix2f0nu/config.yml

You are using toff:0.0.0-alpha3
Parameterize(force_field_code = openff-1.3.1.offxml, ext_types = [mol3 top], hmr_factor = 2.5, overwrite = True, out_dir = /tmp/tmp7ix2f0nu/cmd)
Charges will be corrected: partial_charge - formal_charge = -0.001
After correction: partial_charge - formal_charge = -0.0.


In [42]:
# Check if the file were saved

os.listdir(os.path.join(wd, 'cmd'))

['CMD.top', 'CMD.mol3']